In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sealevel_rise import apply_sea_level, find_underwater_states

In [ ]:
#Start with a simple 7 x 7 test input

topography = np.array([[1,1,0,-1,1,1,1],
                         [1,1,2,3,4,3,1],
                         [2,3,1,0,3,4,3],
                         [1,0,1,1,3,2,3],
                         [0,-1,-2,1,3,3,2],
                         [2,3,-1,1,3,0,-1],
                         [2,3,0,1,3,-1,0]])


In [ ]:
#define variable for sealevel
sealevels = [2]

for sealevel in sealevels:
    new_topography = apply_sea_level(sealevel, topography)
    connected = find_underwater_states(new_topography)
    plt.figure()
    plt.imshow(connected)
